In [101]:
import pandas as pd
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt

In [102]:
clean_data = pd.read_excel("/Users/Justin/GIT/data-question-2/violations2017.xlsx")

In [103]:
clean_data.head(20)

,VIOLATION_ID,ORG_ID,VIOLATION_TYPE,VIOLATION_DESC,VIOLATION_TEXT,REMEDIAL_TEXT,TABLE_NAME,EXPIRED_FLAG,DATE_EXPIRED,CREATED_BY,DATE_CREATED,MODIFIED_BY,DATE_MODIFIED,FEE_SETUP_ID,Unnamed: 14
0,1,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
1,2,1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
2,4,1,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
3,5,1,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
4,6,1,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
5,7,1,BOARDINGRQ,Boarding Requirements,Section 16.24.340 (U) (2) – Boarding of Vacant...,CAAH_BOARDING_REQU,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
6,8,1,BOARDVAC,Boarding of Vacant Buildings,Section 16.24.340 (U) (1) – Boarding of Vacant...,CAAH_BOARDING_VABLDG,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
7,9,1,CARROW,Vehicle In The Right-Of-Way,Section 12.08.210 - Abandoned Vehicles: Abando...,CAMCL_SEW_CONCT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
8,10,1,CERTCOMP,Certificate of Compliance Required,Section 17.40.580 - Certificate of Compliance:...,CAAZ_CERT_COMP,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
9,11,1,CERTOCC,Certificate of Occupancy Required,Section 16.24.130 – Certificate of Occupancy R...,CAAH_CERT_OCC,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN


In [104]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 15 columns):
VIOLATION_ID      928 non-null object
ORG_ID            789 non-null object
VIOLATION_TYPE    788 non-null object
VIOLATION_DESC    925 non-null object
VIOLATION_TEXT    666 non-null object
REMEDIAL_TEXT     345 non-null object
TABLE_NAME        137 non-null datetime64[ns]
EXPIRED_FLAG      651 non-null object
DATE_EXPIRED      2 non-null object
CREATED_BY        650 non-null float64
DATE_CREATED      651 non-null object
MODIFIED_BY       23 non-null object
DATE_MODIFIED     22 non-null datetime64[ns]
FEE_SETUP_ID      0 non-null float64
Unnamed: 14       0 non-null float64
dtypes: datetime64[ns](2), float64(3), object(10)
memory usage: 109.5+ KB


In [105]:
clean_data.iloc[18,0]

19

In [106]:
print(clean_data.iloc[15:18, 4])

15                                                  NaN
16    Section 10.20.330 – Dumpsters – Hours of Colle...
17    Section 16.24.490 (C) - Electrical System Haza...
Name: VIOLATION_TEXT, dtype: object


In [109]:
firstrows = clean_data[:300]

In [144]:
display(firstrows)

,VIOLATION_ID,ORG_ID,VIOLATION_TYPE,VIOLATION_DESC,VIOLATION_TEXT,REMEDIAL_TEXT,TABLE_NAME,EXPIRED_FLAG,DATE_EXPIRED,CREATED_BY,DATE_CREATED,MODIFIED_BY,DATE_MODIFIED,FEE_SETUP_ID,Unnamed: 14,vcount
0,1,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN,NaN
1,2,1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN,NaN
2,4,1,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN,NaN
3,5,1,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN,NaN
4,6,1,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN,NaN
5,7,1,BOARDINGRQ,Boarding Requirements,Section 16.24.340 (U) (2) – Boarding of Vacant...,CAAH_BOARDING_REQU,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN,NaN
6,8,1,BOARDVAC,Boarding of Vacant Buildings,Section 16.24.340 (U) (1) – Boarding of Vacant...,CAAH_BOARDING_VABLDG,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN,NaN
7,9,1,CARROW,Vehicle In The Right-Of-Way,Section 12.08.210 - Abandoned Vehicles: Abando...,CAMCL_SEW_CONCT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN,NaN
8,10,1,CERTCOMP,Certificate of Compliance Required,Section 17.40.580 - Certificate of Compliance:...,CAAZ_CERT_COMP,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN,NaN
9,11,1,CERTOCC,Certificate of Occupancy Required,Section 16.24.130 – Certificate of Occupancy R...,CAAH_CERT_OCC,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN,NaN


In [143]:
##firstrows.vcount = firstrows.VIOLATION_ID.str.len()

/Users/Justin/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [132]:
print(firstrows)

                                             VIOLATION_ID            ORG_ID  \
0                                                       1                 1   
1                                                       2                 1   
2                                                       4                 1   
3                                                       5                 1   
4                                                       6                 1   
5                                                       7                 1   
6                                                       8                 1   
7                                                       9                 1   
8                                                      10                 1   
9                                                      11                 1   
10                                                     12                 1   
11                                                  

In [145]:
if firstrows.VIOLATION_ID.str.len() > 1:
    clean_data['VIOLATION_TEXT'] = str(clean_data['VIOLATION_TEXT']) + str(['clean_data.VIOLATION_ID'])
    
print(clean_data)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().